Словообразовательный словарь

# Русский язык отличается большой свободой в словообразовании. Используя корни и комбинируя приставки и суффиксы, можно создавать новые, раньше не существовавшие слова, которые будут понятны любому (или почти любому) носителю языка. В этой задаче мы предлагаем вам разработать модель, которая будет строить словообразовательный разбор слов, то есть разбивать их на морфемы семи типов: корни (ROOT), приставки (PREF), суффиксы (SUFF), окончания (END), постфиксы (POST), соединительные гласные (LINK) и дефисы (HYPH). Разборы слов представлены в датасете в формате слов:ROOT/о:END, то есть слово разбивается на отдельные морфемы при помощи слэша /, а каждой морфеме приписывается её тип через двоеточие :.

In [ ]:
pip install sklearn_crfsuite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
# from sklearn_crfsuite import CRF
from sklearn.model_selection import train_test_split



In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/Datasets and study/data_For_camp/morpheme_train.csv", header = None)


In [ ]:
words = train_data[0].tolist()
labels = train_data[1].tolist()
def word_to_features(word):
    return [{char : i} for i, char in enumerate(word)]

X = [ list(word) for word in words]
y = [label.split("/") for label in labels]
Y = []
for razmetka in y:
  cur = []
  for metka in razmetka:
    count, val = metka.split(":")
    [cur.append(val) for _ in range(len(count))]
  Y.append(cur)


In [ ]:
crf = CRF(algorithm= 'l2sgd' , max_iterations= 100000)
crf.fit(X, Y)


CRF(algorithm='l2sgd', max_iterations=100000)

In [ ]:
def convert_morpheme_format(word, labels):
    # Удаляем пробелы и разделители, оставляя только последовательность меток

    # Группируем символы слова по одинаковым меткам
    grouped = []
    current_label = labels[0]
    current_chars = word[0]

    for char, label in zip(word[1:], labels[1:]):
        if label == current_label:
            current_chars += char
        else:
            grouped.append((current_chars, current_label))
            current_label = label
            current_chars = char
    grouped.append((current_chars, current_label))  # Добавляем последнюю группу
    # Форматируем результат
    morphemes = "/".join([f"{chars}:{label}" for chars, label in grouped])
    return f"{word},{morphemes}"

In [ ]:
# Предсказание для тестовых данных
test_data = pd.read_csv("/content/drive/MyDrive/Datasets and study/data_For_camp/morpheme_test.csv", header = None)
test_words = test_data[0].tolist()
X_test = [word_to_features(word) for word in test_words]
predicted = crf.predict(X_test)

# Сохранение результатов
with open("submission.csv", "w") as f:
    for word, morphemes in zip(test_words, predicted):
        f.write(convert_morpheme_format(word, morphemes))
        f.write("\n")

[('елизавет', 'ROOT'), ('ински', 'SUFF'), ('й', 'END')]
[('киноф', 'ROOT'), ('икаци', 'SUFF'), ('я', 'END')]
[('хиджр', 'ROOT'), ('а', 'END')]
[('магистерст', 'ROOT'), ('в', 'SUFF'), ('о', 'END')]
[('педант', 'ROOT'), ('ически', 'SUFF'), ('й', 'END')]
[('шелес', 'ROOT'), ('т', 'END')]
[('дегидрат', 'ROOT'), ('аци', 'SUFF'), ('я', 'END')]
[('наряд', 'ROOT'), ('и', 'SUFF'), ('ть', 'END'), ('ся', 'POST')]
[('о', 'PREF'), ('траж', 'ROOT'), ('ени', 'SUFF'), ('е', 'END')]
[('парафраз', 'ROOT')]
[('пере', 'PREF'), ('ст', 'ROOT'), ('а', 'SUFF'), ('ть', 'END')]
[('деб', 'ROOT'), ('а', 'SUFF'), ('ты', 'END')]
[('надлом', 'ROOT'), ('анн', 'SUFF'), ('ый', 'END')]
[('ра', 'PREF'), ('скол', 'ROOT'), ('о', 'SUFF'), ('тый', 'END')]
[('тисн', 'ROOT'), ('и', 'SUFF'), ('ть', 'END')]
[('прищепл', 'ROOT'), ('енн', 'SUFF'), ('ый', 'END')]
[('слом', 'ROOT'), ('а', 'SUFF'), ('ть', 'END'), ('ся', 'POST')]
[('по', 'PREF'), ('набр', 'ROOT'), ('анн', 'SUFF'), ('ый', 'END')]
[('пере', 'PREF'), ('груж', 'ROOT'), ('

# Второй вариант

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(tag2idx)  # Количество классов = размер tag2idx
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased",
                                         do_word_tokenize=False,  # Токенизируем вручную
                                         tokenize_chars=True)    # По с

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
tag2idx = {"PREF": 0, "ROOT": 1, "SUFF": 2, "END": 3}
idx2tag = { 0 : "PREF",  1 :"ROOT",  2 :"SUFF", 3 : "END"}

In [ ]:
word = "отражение"
inputs = tokenizer(list(word), return_tensors="pt", is_split_into_words=True)
outputs = model(**inputs)
predicted_ids = outputs.logits.argmax(-1)[0]  # Индексы меток
predicted_tags = [idx2tag[idx.item()] for idx in predicted_ids]

# # Пример обучения:
# inputs = tokenizer(words,return_tensors="pt", padding=True)
# labels = torch.tensor(labels)
# outputs = model(**inputs, labels=labels)

# Третий

In [ ]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn


In [ ]:
class BiLSTMTagger(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_tags):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim // 2,
                           bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_tags)

    def forward(self, x):
        x = self.embedding(x)  # (batch_size, seq_len, embed_dim)
        x, _ = self.lstm(x)    # (batch_size, seq_len, hidden_dim)
        x = self.fc(x)         # (batch_size, seq_len, num_tags)
        return x

In [ ]:
# Уникальные символы и метки
chars = sorted(set(char for word in train_data[0] for char in word))
tags = ['PREF', 'ROOT', 'SUFF', 'END', 'POST', 'LINK', 'HYPH']

# Словари char2idx и tag2idx
char2idx = {char: idx + 1 for idx, char in enumerate(chars)}  # +1 для padding (0)
tag2idx = {tag: idx for idx, tag in enumerate(tags)}

# Обратные словари
idx2char = {idx: char for char, idx in char2idx.items()}
idx2tag = {idx: tag for tag, idx in tag2idx.items()}

In [ ]:
class MorphemeDataset(Dataset):
    def __init__(self, words, labels, char2idx, tag2idx):
        self.words = words # Сбрасываем индексы
        self.labels = labels
        self.char2idx = char2idx
        self.tag2idx = tag2idx

    def __len__(self):
        return len(self.words)

    def __getitem__(self, idx):
        word = self.words.iloc[idx]  # Используем iloc вместо прямого обращения
        labels = self.labels.iloc[idx]

        # Преобразуем символы и метки в индексы
        x = [self.char2idx[char] for char in word]
        y = [self.tag2idx[tag] for tag in labels]


        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)

def collate_fn(batch):
  # Находим максимальную длину в батче
  max_len = max(len(x[0]) for x in batch)

  # Дополняем последовательности
  x_padded = []
  y_padded = []
  for x, y in batch:
      pad_size = max_len - len(x)
      x_padded.append(torch.cat([x, torch.zeros(pad_size, dtype=torch.long)]))
      y_padded.append(torch.cat([y, torch.zeros(pad_size, dtype=torch.long)]))

  return torch.stack(x_padded), torch.stack(y_padded)


df = pd.DataFrame({
    'chars': X,
    'labels': Y
}).reset_index(drop=True)

train_data, val_data = train_test_split( df  , test_size=0.2, random_state=42)
train_dataset = MorphemeDataset(train_data["chars"], train_data["labels"], char2idx, tag2idx)
val_dataset = MorphemeDataset(val_data["chars"], val_data["labels"], char2idx, tag2idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BiLSTMTagger(
    vocab_size=len(char2idx) + 1,  # +1 для padding
    embed_dim=64,
    hidden_dim=128,
    num_tags=len(tag2idx)
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=-1)  # Игнорируем padding
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    for batch in dataloader:
        x, y = batch
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs.view(-1, outputs.shape[-1]), y.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)



In [ ]:
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            x, y = batch
            x, y = x.to(device), y.to(device)

            outputs = model(x)
            loss = criterion(outputs.view(-1, outputs.shape[-1]), y.view(-1))
            total_loss += loss.item()

            preds = outputs.argmax(-1)
            correct += (preds == y).sum().item()
            total += y.numel()

    accuracy = correct / total
    avg_loss = total_loss / len(dataloader)
    return avg_loss, accuracy

In [ ]:
def predict(model, word, char2idx, idx2tag):
    model.eval()
    chars = list(word)
    x = [char2idx.get(char, 0) for char in chars]  # 0 для неизвестных символов
    x = torch.tensor([x], dtype=torch.long).to(device)

    with torch.no_grad():
        outputs = model(x)
        pred_ids = outputs.argmax(-1)[0].cpu().numpy()

    tags = [idx2tag[idx] for idx in pred_ids]
    return tags

In [ ]:
# Обучение
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    print(f"Epoch {epoch + 1}, Loss: {train_loss:.4f}")

Epoch 1, Loss: 0.2208
Epoch 2, Loss: 0.1153
Epoch 3, Loss: 0.0988
Epoch 4, Loss: 0.0884
Epoch 5, Loss: 0.0803
Epoch 6, Loss: 0.0733
Epoch 7, Loss: 0.0676
Epoch 8, Loss: 0.0624
Epoch 9, Loss: 0.0578
Epoch 10, Loss: 0.0534


In [ ]:
val_loss, val_acc = evaluate(model, val_loader, criterion)
print(f"Валидационная точность: {val_acc:.2%}")

In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/Datasets and study/data_For_camp/morpheme_test.csv", header = None)
test_words = test_data[0].tolist()
X_test = [list(word) for word in test_words]
predicted = []
for word in X_test:
   predicted.append(predict(model, word, char2idx, idx2tag))



In [ ]:
predicted

[['ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'SUFF',
  'SUFF',
  'SUFF',
  'SUFF',
  'END',
  'END'],
 ['ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'SUFF',
  'SUFF',
  'SUFF',
  'SUFF',
  'SUFF',
  'END'],
 ['ROOT', 'ROOT', 'ROOT', 'ROOT', 'SUFF', 'END'],
 ['ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'SUFF',
  'SUFF',
  'SUFF',
  'SUFF',
  'SUFF',
  'END'],
 ['ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'SUFF',
  'SUFF',
  'SUFF',
  'SUFF',
  'SUFF',
  'END',
  'END'],
 ['ROOT', 'ROOT', 'ROOT', 'ROOT', 'ROOT', 'ROOT'],
 ['ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'SUFF',
  'SUFF',
  'SUFF',
  'SUFF',
  'SUFF',
  'END'],
 ['ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'ROOT',
  'SUFF',
  'END',
  'END',
  'POST',
  'POST'],
 ['ROOT', 'ROOT', 'ROOT', 'ROOT', 'ROOT', 'SUFF', 'SUFF', 'SUFF', 'END'],
 ['ROOT', 'ROOT', 'ROOT', 'ROOT', 'ROOT', 'ROOT', 'ROOT', 'ROOT'],
 ['PREF', 'PREF', 'ROOT', 'ROOT', 'ROOT',

In [ ]:
# Сохранение результатов
with open("submission.csv", "w") as f:
    for word, morphemes in zip(test_words, predicted):
        f.write(convert_morpheme_format(word, morphemes))
        f.write("\n")